In [12]:
import json
timerSum = {"ch":{"su":{"p1":{"on":"10:00","off":"10:30"}}}}
outfile=open("myfile.json","w")
json.dump(timerSum, outfile, indent=6)

t={}
for c in range(4):
  kc="c"+ str(c)
  t[kc]={}
  for d in range(7):
    kd="d"+str(d)
    t[kc][kd]={}
    for p in range(4) :
      kp="p"+str(p)
      t[kc][kd][kp]={"on":"","off":""}

pj = json.dumps(t, indent=4)
print(pj)
print(t)

outfile.close()

{
    "c0": {
        "d0": {
            "p0": {
                "on": "",
                "off": ""
            },
            "p1": {
                "on": "",
                "off": ""
            },
            "p2": {
                "on": "",
                "off": ""
            },
            "p3": {
                "on": "",
                "off": ""
            }
        },
        "d1": {
            "p0": {
                "on": "",
                "off": ""
            },
            "p1": {
                "on": "",
                "off": ""
            },
            "p2": {
                "on": "",
                "off": ""
            },
            "p3": {
                "on": "",
                "off": ""
            }
        },
        "d2": {
            "p0": {
                "on": "",
                "off": ""
            },
            "p1": {
                "on": "",
                "off": ""
            },
            "p2": {
                "on": "",
  

In [4]:
import json
f=open('params.json',)
data = json.load(f)

print(data)
# for i in data['ch']:
#     print(i)

f.close()

{'data': {'c1': {'su': {'p1': {'on': '20:25', 'off': '22:48'}, 'p2': {'on': '22:12', 'off': '10:36'}, 'p3': {'on': '15:56', 'off': '11:29'}, 'p4': {'on': '21:15', 'off': '10:45'}}, 'mo': {'p1': {'on': '10:12', 'off': '12:19'}, 'p2': {'on': '22:57', 'off': '20:48'}, 'p3': {'on': '15:49', 'off': '23:13'}, 'p4': {'on': '19:51', 'off': '11:40'}}, 'tu': {'p1': {'on': '18:32', 'off': '21:58'}, 'p2': {'on': '18:39', 'off': '18:21'}, 'p3': {'on': '15:19', 'off': '20:34'}, 'p4': {'on': '21:51', 'off': '10:24'}}, 'we': {'p1': {'on': '22:41', 'off': '22:35'}, 'p2': {'on': '19:32', 'off': '17:23'}, 'p3': {'on': '11:24', 'off': '22:55'}, 'p4': {'on': '15:25', 'off': '22:24'}}, 'th': {'p1': {'on': '14:16', 'off': '20:10'}, 'p2': {'on': '19:20', 'off': '17:50'}, 'p3': {'on': '22:47', 'off': '14:53'}, 'p4': {'on': '18:47', 'off': '11:32'}}, 'fr': {'p1': {'on': '19:52', 'off': '18:27'}, 'p2': {'on': '15:45', 'off': '16:38'}, 'p3': {'on': '13:42', 'off': '21:30'}, 'p4': {'on': '10:47', 'off': '12:43'}},

In [1]:
from flask import Flask
from flask.templating import render_template
app = Flask('app')

@app.route('/')
def hello_world():
  return render_template('hello.html')

@app.route('/blogs/<int:blog_id>')
def blogs(blog_id):
  return render_template('blogs.html', blog_id=blog_id)  
  
app.run(host='0.0.0.0', port=8080)


 * Serving Flask app 'app'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://192.168.1.115:8080
Press CTRL+C to quit
127.0.0.1 - - [11/Oct/2023 16:40:21] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Oct/2023 16:40:21] "GET /favicon.ico HTTP/1.1" 404 -


In [48]:
from flask import Flask, render_template, request, jsonify
import json
from random import *
from math import *

import ntplib
from time import ctime

def get_ntp_time():
    c = ntplib.NTPClient()
    try:
        response = c.request('pool.ntp.org', version=3)
        return response.tx_time
    except ntplib.NTPException as e:
        print(f"NTPException: {e}")
        return None

app = Flask('app')

@app.route('/getTimerSetData')
def get_timerSetData():
    with open("params5.json") as f:
        data = json.load(f)
        return data

@app.route('/get_time')
def request_time():
    ntp_time = get_ntp_time()
    return jsonify({"time": ntp_time})

# @app.route('/')
# def homepage():
#     return render_template("form.html", len=4)

@app.route('/data/', methods=['POST','GET'])
def data():
    if request.method == 'GET':
        with open("params2.json") as f:
            data = json.load(f)
            print(data)
            return data
        return f"The URL /data is accessed directly Try going to '/form' to submit form."
    if request.method == 'POST':
        form_data = request.get_json()
        print("post")
        print(request.get_json())

        f = open("params2.json", "w")
        f.write(json.dumps(request.get_json(), indent=4))
        f.close()

        return render_template('data.html', form_data=form_data)
myform=[]
# @app.route('/jsondata/', methods=['GET','POST'])
@app.route('/', methods=['GET','POST'])
def jsondata():
    # ntptime = request_time()
    ntp_time = get_ntp_time()
    print("ntptime=", ntp_time)
    if request.method == 'GET':
        print("GET")
        # params3.json (old format that worked with populateDataFull). params5.json works with populateDataFull2
        with open("params5.json") as f:
            jsondata = json.load(f)
            # print(jsondata)
            return render_template('jsondata.html', jsondata = jsondata, ntp_time = ntp_time)

    if request.method == 'POST':
        print('POST')

        try:
            data = request.get_json()
            print(data)

            with open("params5.json", "w") as f:
                f.write(json.dumps(request.get_json(), indent=4))

            with open("params5.json") as f:
                jsondata = json.load(f)
                return jsonify(jsondata=jsondata, ntptime=ntp_time)
        except Exception as e:
            return jsonify({"message": str(e)})
        print('POST')
        myform=request.form
        print(request.form)
        return '<h1>successfully posted</h1>'

@app.route('/getdata')
def getdata():
    ntp_time = get_ntp_time()
    with open("params5.json") as f:
        jsondata = json.load(f)
        return jsonify(jsondata=jsondata, ntptime = ntp_time)
    
@app.route('/postdata', methods=['GET','POST'])
def postdata():
    if request.method == 'POST':
        form_data = request.form
        print(form_data)
        # print("post")
        # print(request.get_json())
        with open("params4.json", "w") as f:
            f.write(json.dumps(form_data))
        return "<p>results saved in params4.json</p>"
        
@app.route('/generatedata/', methods=['GET'])
def generatedata():
    data={}
    carr = []
    for c in range(8):
        darr=[]
        for d in range(7):
            parr=[]
            for p in range(4):
                parr.append(generateTime())
            darr.append(parr)
        carr.append(darr)
    data["data"] = carr
    with open('params3.json',"w") as f:
        f.write(json.dumps(data))
    return "<p>results generated in params3.json</p>"

def generateTime():
    t0 = twoDigit(24) + ":" + twoDigit(60)
    t1 = twoDigit(24) + ":" + twoDigit(60)
    tarr=[]
    tarr.append(t0)
    tarr.append(t1)
    return tarr

def twoDigit(r):
    t=floor(random()*r)
    t=str(int(t))
    t=t.zfill(2)
    return t

app.run(host='0.0.0.0', port=8080)

 * Serving Flask app 'app'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://192.168.1.115:8080
Press CTRL+C to quit
192.168.1.115 - - [14/Nov/2023 16:04:18] "GET /jsondata/ HTTP/1.1" 200 -
192.168.1.115 - - [14/Nov/2023 16:04:18] "GET /static/css/style.css HTTP/1.1" 200 -
192.168.1.115 - - [14/Nov/2023 16:04:18] "GET /static/js/script.js HTTP/1.1" 200 -


ntptime= 1700006658.2351727
GET


192.168.1.115 - - [14/Nov/2023 16:04:28] "GET /get_time HTTP/1.1" 200 -
192.168.1.115 - - [14/Nov/2023 16:04:38] "GET /get_time HTTP/1.1" 200 -
192.168.1.115 - - [14/Nov/2023 16:04:48] "GET /get_time HTTP/1.1" 200 -
192.168.1.115 - - [14/Nov/2023 16:04:58] "GET /get_time HTTP/1.1" 200 -
192.168.1.115 - - [14/Nov/2023 16:05:08] "GET /get_time HTTP/1.1" 200 -
192.168.1.115 - - [14/Nov/2023 16:05:18] "GET /get_time HTTP/1.1" 200 -
192.168.1.115 - - [14/Nov/2023 16:05:28] "GET /get_time HTTP/1.1" 200 -
192.168.1.115 - - [14/Nov/2023 16:05:38] "GET /get_time HTTP/1.1" 200 -
192.168.1.115 - - [14/Nov/2023 16:05:48] "GET /get_time HTTP/1.1" 200 -
192.168.1.115 - - [14/Nov/2023 16:05:59] "GET /get_time HTTP/1.1" 200 -
192.168.1.115 - - [14/Nov/2023 16:06:09] "GET /get_time HTTP/1.1" 200 -
192.168.1.115 - - [14/Nov/2023 16:06:20] "GET /get_time HTTP/1.1" 200 -
192.168.1.115 - - [14/Nov/2023 16:06:30] "GET /get_time HTTP/1.1" 200 -
192.168.1.115 - - [14/Nov/2023 16:06:40] "GET /get_time HTTP/1.1

NTPException: No response received from pool.ntp.org.


192.168.1.115 - - [14/Nov/2023 16:09:23] "GET /get_time HTTP/1.1" 200 -
192.168.1.115 - - [14/Nov/2023 16:09:33] "GET /get_time HTTP/1.1" 200 -
192.168.1.115 - - [14/Nov/2023 16:09:43] "GET /get_time HTTP/1.1" 200 -
192.168.1.115 - - [14/Nov/2023 16:09:53] "GET /get_time HTTP/1.1" 200 -
192.168.1.115 - - [14/Nov/2023 16:10:03] "GET /get_time HTTP/1.1" 200 -
192.168.1.115 - - [14/Nov/2023 16:10:14] "GET /get_time HTTP/1.1" 200 -
192.168.1.115 - - [14/Nov/2023 16:10:24] "GET /get_time HTTP/1.1" 200 -
192.168.1.115 - - [14/Nov/2023 16:10:34] "GET /get_time HTTP/1.1" 200 -
192.168.1.115 - - [14/Nov/2023 16:10:44] "GET /get_time HTTP/1.1" 200 -
192.168.1.115 - - [14/Nov/2023 16:10:54] "GET /get_time HTTP/1.1" 200 -
192.168.1.115 - - [14/Nov/2023 16:11:05] "GET /get_time HTTP/1.1" 200 -
192.168.1.115 - - [14/Nov/2023 16:11:15] "GET /get_time HTTP/1.1" 200 -
192.168.1.115 - - [14/Nov/2023 16:11:25] "GET /get_time HTTP/1.1" 200 -
192.168.1.115 - - [14/Nov/2023 16:11:35] "GET /get_time HTTP/1.1

NTPException: No response received from pool.ntp.org.


192.168.1.115 - - [14/Nov/2023 16:20:43] "GET /get_time HTTP/1.1" 200 -
192.168.1.115 - - [14/Nov/2023 16:20:54] "GET /get_time HTTP/1.1" 200 -
192.168.1.115 - - [14/Nov/2023 16:21:04] "GET /get_time HTTP/1.1" 200 -
192.168.1.115 - - [14/Nov/2023 16:21:14] "GET /get_time HTTP/1.1" 200 -
192.168.1.115 - - [14/Nov/2023 16:21:24] "GET /get_time HTTP/1.1" 200 -
192.168.1.115 - - [14/Nov/2023 16:21:34] "GET /get_time HTTP/1.1" 200 -
192.168.1.115 - - [14/Nov/2023 16:21:44] "GET /get_time HTTP/1.1" 200 -
192.168.1.115 - - [14/Nov/2023 16:21:54] "GET /get_time HTTP/1.1" 200 -
192.168.1.115 - - [14/Nov/2023 16:22:04] "GET /get_time HTTP/1.1" 200 -
192.168.1.115 - - [14/Nov/2023 16:22:15] "GET /get_time HTTP/1.1" 200 -
192.168.1.115 - - [14/Nov/2023 16:22:25] "GET /get_time HTTP/1.1" 200 -
192.168.1.115 - - [14/Nov/2023 16:22:35] "GET /get_time HTTP/1.1" 200 -
192.168.1.115 - - [14/Nov/2023 16:22:45] "GET /get_time HTTP/1.1" 200 -
192.168.1.115 - - [14/Nov/2023 16:22:55] "GET /get_time HTTP/1.1

In [141]:
def myMap(i):
    ii=4*(i-1)
    return 1+ii%20+floor(ii/20)

for x in range(1,21):
    print(f'{x}, {myMap(x)}')

1, 1
2, 5
3, 9
4, 13
5, 17
6, 2
7, 6
8, 10
9, 14
10, 18
11, 3
12, 7
13, 11
14, 15
15, 19
16, 4
17, 8
18, 12
19, 16
20, 20


In [136]:
y=10
f'he{y}'
'{}'.format(y)
for y in range(3):
    f'hello {y}'

In [1]:
from flask import Flask, render_template, jsonify

app = Flask('app')
mydata=''
@app.route('/atticus/')
def atticus():
    return 'hello atticus'

# SPA example 1: https://github.com/praveen-me/simple-vanila-router
@app.route('/')
def problem1():
    return render_template('problem1.html')

# SPA example 2: https://github.com/zathio/vanilla-spa
@app.route('/spa')
def problem2():
    return render_template('problem2.html')

@app.route('/data/', methods=['GET','POST'])
def problem3():
    if request.method == 'GET':
        return render_template('problem3.html')
    try:
        data = request.get_json()
        print(data)
        data2 = {"message":"bye"}
        return jsonify({"data": data2})
    except Exception as e:
        return jsonify({"message": str(e)})
    print(request.form)
    mydata=request.form
    return render_template('problem3.html')

app.run(host='0.0.0.0', port=8081)

KeyboardInterrupt: 

In [51]:
print(mydata.data)

AttributeError: 'str' object has no attribute 'data'

In [3]:
print(myform)

[]


In [110]:
for i in range(100):
    print('{} {}' , (i,i*i*i))

{} {} (0, 0)
{} {} (1, 1)
{} {} (2, 8)
{} {} (3, 27)
{} {} (4, 64)
{} {} (5, 125)
{} {} (6, 216)
{} {} (7, 343)
{} {} (8, 512)
{} {} (9, 729)
{} {} (10, 1000)
{} {} (11, 1331)
{} {} (12, 1728)
{} {} (13, 2197)
{} {} (14, 2744)
{} {} (15, 3375)
{} {} (16, 4096)
{} {} (17, 4913)
{} {} (18, 5832)
{} {} (19, 6859)
{} {} (20, 8000)
{} {} (21, 9261)
{} {} (22, 10648)
{} {} (23, 12167)
{} {} (24, 13824)
{} {} (25, 15625)
{} {} (26, 17576)
{} {} (27, 19683)
{} {} (28, 21952)
{} {} (29, 24389)
{} {} (30, 27000)
{} {} (31, 29791)
{} {} (32, 32768)
{} {} (33, 35937)
{} {} (34, 39304)
{} {} (35, 42875)
{} {} (36, 46656)
{} {} (37, 50653)
{} {} (38, 54872)
{} {} (39, 59319)
{} {} (40, 64000)
{} {} (41, 68921)
{} {} (42, 74088)
{} {} (43, 79507)
{} {} (44, 85184)
{} {} (45, 91125)
{} {} (46, 97336)
{} {} (47, 103823)
{} {} (48, 110592)
{} {} (49, 117649)
{} {} (50, 125000)
{} {} (51, 132651)
{} {} (52, 140608)
{} {} (53, 148877)
{} {} (54, 157464)
{} {} (55, 166375)
{} {} (56, 175616)
{} {} (57, 185

In [24]:
import ntplib
from time import ctime
c = ntplib.NTPClient()
response = c.request('pool.ntp.org', version=3)
ctime(response.tx_time)

'Thu Oct 26 19:01:32 2023'

In [27]:
import ntplib
from time import ctime
c = ntplib.NTPClient()
response = c.request('pool.ntp.org', version=3)
ctime(response.tx_time)

'Thu Oct 26 19:12:45 2023'

In [18]:
from math import pi, pow
def findM(T,R1,R2):
    d=R1+R2
    G=6.673e-11 
    s=1.991e30  # sun mass, kg
    k=pow(2*pi/T*d,2)/G
    m1=k*R2
    m2=k*R1
    return (m1,m2,m1/s,m2/s)

T=40*365*24*3600 # 40 years
R1=6e11
R2=9e11
(m1,m2,m1_s,m2_s) = findM(T,R1,R2)
print(f'm1={m1:.3e}kg or {m1_s:.3f} solar mass, m2={m2:.3e}kg or {m2_s:.3f} solar mass')

m1=7.529e+29kg or 0.378 solar mass, m2=5.019e+29kg or 0.252 solar mass


In [52]:
jsondata= {'timerSetData': {'c1d5p0o0': '13:00', 'c1d5p0o1': '14:00'}}
ntptime={'ntptime':'12345'}
jsondata.update(ntptime)
print(jsondata)

{'timerSetData': {'c1d5p0o0': '13:00', 'c1d5p0o1': '14:00'}, 'ntptime': '12345'}
